# Here a section for pre-processing data scraped

In [ ]:
import json
import pandas as pd
from datetime import datetime
import spacy


In [ ]:
# Charger le fichier JSON
with open("results.json", "r", encoding="utf-8") as f:
    data = json.load(f)

rows = []

for publication in data:
    # Nettoyage du texte de la publication
    post_text = "\n".join([p.strip() for p in publication.get("post", []) if p.strip()])
    
    # Format date (facultatif)
    raw_date = publication.get("date_publication", "").replace("Publié le ", "")
    
    # Ajouter la publication principale
    rows.append({
        "type": "publication",
        "date": raw_date,
        "texte": post_text,
        "source_url": publication.get("url", "")
    })

    # Ajouter les commentaires et replies
    for comment in publication.get("comments", []):
        comment_text = comment.get("text", "").strip()
        comment_date = comment.get("date", "").strip()
        
        rows.append({
            "type": "comment",
            "date": comment_date,
            "texte": comment_text,
            "source_url": publication.get("url", "")
        })

        for reply in comment.get("replies", []):
            reply_text = reply.get("text", "").strip()
            reply_date = reply.get("date", "").strip()
            
            rows.append({
                "type": "reply",
                "date": reply_date,
                "texte": reply_text,
                "source_url": publication.get("url", "")
            })

# Convertir en DataFrame
df = pd.DataFrame(rows)

In [2]:
df

,type,date,texte,source_url
0,publication,jeudi 3 avril 2025 à 23h12min,La Coordination nationale des associations de ...,https://lefaso.net/spip.php?article137183
1,comment,4 avril 09:40,C’est la photo de famille entre donnateurs et ...,https://lefaso.net/spip.php?article137183
2,publication,mardi 1er avril 2025 à 22h30min,"1. Bien s’hydrater\nBoire au moins 1,5 à 2 lit...",https://lefaso.net/spip.php?article137125
3,publication,samedi 29 mars 2025 à 21h51min,"François Sama a soutenu, ce vendredi 28 mars 2...",https://lefaso.net/spip.php?article137081
4,comment,30 mars 09:30,"Une thèse de philosophie sur Spinoza pour ""con...",https://lefaso.net/spip.php?article137081
...,...,...,...,...
5016,comment,24 décembre 2024 09:16,"Suite à ce discours extrêmement déterminé, on ...",https://lefaso.net/spip.php?article135011
5017,comment,25 décembre 2024 00:54,Tous les discours tendant à disculper la Franc...,https://lefaso.net/spip.php?article135011
5018,reply,25 décembre 2024 21:41,Difficile de comprendre ton idée car va dans t...,https://lefaso.net/spip.php?article135011
5019,comment,27 décembre 2024 08:19,"@EcowasBoy, vous n’êtes et vous ne serez jamai...",https://lefaso.net/spip.php?article135011


## I-Pre-Processing

In [3]:
!pip install spacy

In [4]:
!python -m spacy download fr_core_news_sm

     ---------------------------------------- 0.0/16.3 MB ? eta -:--:--
      --------------------------------------- 0.3/16.3 MB 6.7 MB/s eta 0:00:03
     - -------------------------------------- 0.7/16.3 MB 8.9 MB/s eta 0:00:02
     --- ------------------------------------ 1.6/16.3 MB 12.7 MB/s eta 0:00:02
     ----- ---------------------------------- 2.4/16.3 MB 12.9 MB/s eta 0:00:02
     ----- ---------------------------------- 2.4/16.3 MB 12.9 MB/s eta 0:00:02
     --------- ------------------------------ 3.8/16.3 MB 13.4 MB/s eta 0:00:01
     ------------ --------------------------- 5.0/16.3 MB 15.2 MB/s eta 0:00:01
     ------------ --------------------------- 5.1/16.3 MB 14.9 MB/s eta 0:00:01
     -------------- ------------------------- 5.8/16.3 MB 14.2 MB/s eta 0:00:01
     -------------- ------------------------- 5.8/16.3 MB 14.2 MB/s eta 0:00:01
     -------------- ------------------------- 5.8/16.3 MB 14.2 MB/s eta 0:00:01
     -------------- ------------------------- 5.8/

In [7]:
# Model loading
nlp = spacy.load("fr_core_news_sm")

In [8]:
def preprocess_text(text):
    doc = nlp(text)

    tokens = [
        token.lemma_.lower() 
        for token in doc 
        if not token.is_stop and not token.is_punct and not token.is_space
    ]
    
    return " ".join(tokens)

In [9]:
df["text_processed"] = df["texte"].apply(preprocess_text) #preprocess of text

In [10]:
df

,type,date,texte,source_url,text_processed
0,publication,jeudi 3 avril 2025 à 23h12min,La Coordination nationale des associations de ...,https://lefaso.net/spip.php?article137183,coordination national association veille citoy...
1,comment,4 avril 09:40,C’est la photo de famille entre donnateurs et ...,https://lefaso.net/spip.php?article137183,photo famille donnateur beneficiaire vraiment ...
2,publication,mardi 1er avril 2025 à 22h30min,"1. Bien s’hydrater\nBoire au moins 1,5 à 2 lit...",https://lefaso.net/spip.php?article137125,"1 bien hydrater boire 1,5 2 litre eau jour cas..."
3,publication,samedi 29 mars 2025 à 21h51min,"François Sama a soutenu, ce vendredi 28 mars 2...",https://lefaso.net/spip.php?article137081,françois sama soutenir vendredi 28 mars 2025 u...
4,comment,30 mars 09:30,"Une thèse de philosophie sur Spinoza pour ""con...",https://lefaso.net/spip.php?article137081,thèse philosophie spinoza contribuer pacificat...
...,...,...,...,...,...
5016,comment,24 décembre 2024 09:16,"Suite à ce discours extrêmement déterminé, on ...",https://lefaso.net/spip.php?article135011,suite discours extrêmement déterminé attendre ...
5017,comment,25 décembre 2024 00:54,Tous les discours tendant à disculper la Franc...,https://lefaso.net/spip.php?article135011,discours tendre disculper france état satellit...
5018,reply,25 décembre 2024 21:41,Difficile de comprendre ton idée car va dans t...,https://lefaso.net/spip.php?article135011,difficile comprendre ider sens budget faso def...
5019,comment,27 décembre 2024 08:19,"@EcowasBoy, vous n’êtes et vous ne serez jamai...",https://lefaso.net/spip.php?article135011,@ecowasboy être être jamais référence aucun sc...


#### Normalization of Date to ISO format

In [11]:
!pip install dateparser

In [18]:
import datetime
import dateparser
import re

def normaliser_date(date_str, reference_year=None):
    # Nettoyage manuel
    clean_date = date_str.lower()
    clean_date = re.sub(r'publié le ', '', clean_date)
    clean_date = re.sub(r'à ', '', clean_date)
    clean_date = re.sub(r'min', '', clean_date)
    clean_date = clean_date.strip()

    # Settings pour dateparser
    settings = {'PREFER_DATES_FROM': 'past', 'DATE_ORDER': 'DMY'}

    if reference_year:
        settings['RELATIVE_BASE'] = datetime.datetime(reference_year, 1, 1)

    parsed = dateparser.parse(clean_date, settings=settings)

    if parsed:
        return parsed.strftime("%Y-%m-%d %H:%M:%S")
    else:
        return None

In [19]:
# Application à une série de dates
df["date_normalized"] = df["date"].apply(lambda x: normaliser_date(x, reference_year=2025))

In [20]:
df

,type,date,texte,source_url,text_processed,date_normalized
0,publication,jeudi 3 avril 2025 à 23h12min,La Coordination nationale des associations de ...,https://lefaso.net/spip.php?article137183,coordination national association veille citoy...,2025-04-03 23:12:00
1,comment,4 avril 09:40,C’est la photo de famille entre donnateurs et ...,https://lefaso.net/spip.php?article137183,photo famille donnateur beneficiaire vraiment ...,2024-04-04 09:40:00
2,publication,mardi 1er avril 2025 à 22h30min,"1. Bien s’hydrater\nBoire au moins 1,5 à 2 lit...",https://lefaso.net/spip.php?article137125,"1 bien hydrater boire 1,5 2 litre eau jour cas...",2025-04-01 22:30:00
3,publication,samedi 29 mars 2025 à 21h51min,"François Sama a soutenu, ce vendredi 28 mars 2...",https://lefaso.net/spip.php?article137081,françois sama soutenir vendredi 28 mars 2025 u...,2025-03-29 21:51:00
4,comment,30 mars 09:30,"Une thèse de philosophie sur Spinoza pour ""con...",https://lefaso.net/spip.php?article137081,thèse philosophie spinoza contribuer pacificat...,2024-03-30 09:30:00
...,...,...,...,...,...,...
5016,comment,24 décembre 2024 09:16,"Suite à ce discours extrêmement déterminé, on ...",https://lefaso.net/spip.php?article135011,suite discours extrêmement déterminé attendre ...,2024-12-24 09:16:00
5017,comment,25 décembre 2024 00:54,Tous les discours tendant à disculper la Franc...,https://lefaso.net/spip.php?article135011,discours tendre disculper france état satellit...,2024-12-25 00:54:00
5018,reply,25 décembre 2024 21:41,Difficile de comprendre ton idée car va dans t...,https://lefaso.net/spip.php?article135011,difficile comprendre ider sens budget faso def...,2024-12-25 21:41:00
5019,comment,27 décembre 2024 08:19,"@EcowasBoy, vous n’êtes et vous ne serez jamai...",https://lefaso.net/spip.php?article135011,@ecowasboy être être jamais référence aucun sc...,2024-12-27 08:19:00


In [21]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5021 entries, 0 to 5020
Data columns (total 6 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   type             5021 non-null   object
 1   date             5021 non-null   object
 2   texte            5021 non-null   object
 3   source_url       5021 non-null   object
 4   text_processed   5021 non-null   object
 5   date_normalized  5021 non-null   object
dtypes: object(6)
memory usage: 235.5+ KB


In [22]:
df.to_csv("data_processed/dataset_nlp.csv", index=False, encoding="utf-8")